In [ ]:
def process_user_input(user_input, tfidf, tfidf_matrix, filtered_data, nnp_and_nng_review):
    """
    사용자 입력을 처리하고 유사한 영화를 추천합니다.
    
    Parameters:
        user_input (str): 사용자가 입력한 텍스트
        tfidf (TfidfVectorizer): TF-IDF 벡터화 객체
        tfidf_matrix (csr_matrix): 전체 영화 데이터의 TF-IDF 매트릭스
        filtered_data (DataFrame): 추천을 위한 필터링된 영화 데이터
        nnp_and_nng_review (list): 리뷰에서 추출된 명사 및 명사구 리스트
    
    Returns:
        list: 추천된 영화 목록과 장르 (최대 5개)
        list: 각 영화의 유사도 점수
    """
    sim_scores = []
    print("nnp_and_nng_review 내용:", nnp_and_nng_review)
    
    if contains_content_or_summary(nnp_and_nng_review):
        user_tfidf_vector = vectorize_user_input(user_input, tfidf)
        cosine_sim_user = calculate_cosine_similarity(user_tfidf_vector, tfidf_matrix)
        sim_scores = calculate_similarity_scores(cosine_sim_user, filtered_data)
        
        # 상위 5개의 추천 결과 반환
        recommendations, similarity_scores = get_top_recommendations(sim_scores, filtered_data)
        return recommendations, similarity_scores
    
    else:
        print('내용 또는 줄거리가 포함되지 않은 경우 처리 로직이 필요합니다.')
        return [], []


In [ ]:
def contains_content_or_summary(nnp_and_nng_review):
    """
    리뷰에 '내용' 또는 '줄거리' 단어가 포함되어 있는지 확인합니다.
    
    Parameters:
        nnp_and_nng_review (list): 리뷰에서 추출된 명사 및 명사구 리스트
    
    Returns:
        bool: '내용' 또는 '줄거리' 단어가 포함되어 있으면 True, 그렇지 않으면 False
    """
    return any(word == '내용' or word == '줄거리' for word, tag in nnp_and_nng_review)


In [ ]:
def vectorize_user_input(user_input, tfidf):
    """
    사용자의 입력을 벡터화합니다.
    
    Parameters:
        user_input (str): 사용자가 입력한 텍스트
        tfidf (TfidfVectorizer): TF-IDF 벡터화 객체
    
    Returns:
        csr_matrix: 벡터화된 사용자 입력
    """
    return tfidf.transform([tokenize_and_lemmatize(user_input)])


In [ ]:
def calculate_cosine_similarity(user_tfidf_vector, tfidf_matrix):
    """
    사용자 입력 벡터와 전체 영화 데이터 벡터 간의 코사인 유사도를 계산합니다.
    
    Parameters:
        user_tfidf_vector (csr_matrix): 벡터화된 사용자 입력
        tfidf_matrix (csr_matrix): 전체 영화 데이터의 TF-IDF 매트릭스
    
    Returns:
        np.ndarray: 코사인 유사도 배열
    """
    return linear_kernel(user_tfidf_vector, tfidf_matrix)


In [ ]:
def calculate_similarity_scores(cosine_sim_user, filtered_data):
    """
    코사인 유사도에 기반하여 유사도 점수를 계산합니다.
    
    Parameters:
        cosine_sim_user (np.ndarray): 사용자 입력과 영화 데이터 간의 코사인 유사도
        filtered_data (DataFrame): 추천을 위한 필터링된 영화 데이터
    
    Returns:
        list: 각 영화의 인덱스와 유사도 점수
    """
    sim_scores = []
    for idx in filtered_data.index:
        sim_score = cosine_sim_user[0, idx]
        total_score = sim_score  # 필요한 가중치 계산을 추가할 수 있습니다.
        sim_scores.append((idx, total_score))
    return sim_scores


In [ ]:
def get_top_recommendations(sim_scores, data):
    """
    유사도 점수에 따라 상위 5개의 영화를 추천합니다.
    
    Parameters:
        sim_scores (list): 각 영화의 인덱스와 유사도 점수 리스트
        data (DataFrame): 영화 데이터
    
    Returns:
        list: 추천된 영화 목록과 장르
        list: 각 영화의 유사도 점수
    """
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[:5]
    movie_indices = [i[0] for i in sim_scores]
    similarity_scores = [i[1] for i in sim_scores]
    recommendations = data.iloc[movie_indices][['영화명', '장르']].values.tolist()
    return recommendations, similarity_scores


In [ ]:
# Assuming tokenize_and_lemmatize function was defined or imported before.
# If it's not defined, it should be implemented or imported accordingly.


In [ ]:
# Example usage:
user_input = '...'  # user input string
nnp_and_nng_review = [...]  # list of extracted nouns and noun phrases from reviews
tfidf = ...  # TF-IDF vectorizer object
tfidf_matrix = ...  # TF-IDF matrix of the entire movie dataset
filtered_data = ...  # filtered movie data for recommendations
recommendations, similarity_scores = process_user_input(user_input, tfidf, tfidf_matrix, filtered_data, nnp_and_nng_review)
print(recommendations)
print(similarity_scores)
